<a href="https://colab.research.google.com/github/zp2226/E4511-2021-Zhanbo/blob/main/Data_Clean_%26_Merge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 1.x

# get the Anaconda file 
! wget -c https://repo.anaconda.com/archive/Anaconda3-2019.10-Linux-x86_64.sh
! chmod +x Anaconda3-2019.10-Linux-x86_64.sh
! bash ./Anaconda3-2019.10-Linux-x86_64.sh -b -f -p /usr/local

! time conda install -y -c deepchem -c rdkit -c conda-forge -c omnia deepchem-gpu=2.3.0

import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')
import deepchem as dc

# install mordred, bravado and molvs
! time conda install -c conda-forge -y mordred bravado molvs

# get the Install AMPL_GPU_test.sh
!wget https://raw.githubusercontent.com/ravichas/AMPL-Tutorial/master/config/install_AMPL_GPU_test.sh

# run the script to install AMPL
! chmod u+x install_AMPL_GPU_test.sh
! ./install_AMPL_GPU_test.sh

TensorFlow 1.x selected.
--2021-04-28 02:59:01--  https://repo.anaconda.com/archive/Anaconda3-2019.10-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.131.3, 104.16.130.3, 2606:4700::6810:8303, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.131.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 530308481 (506M) [application/x-sh]
Saving to: ‘Anaconda3-2019.10-Linux-x86_64.sh’

Anaconda3-2019.10-L 100%[===================>] 505.74M   118MB/s    in 4.3s    

2021-04-28 02:59:06 (118 MB/s) - ‘Anaconda3-2019.10-Linux-x86_64.sh’ saved [530308481/530308481]

PREFIX=/usr/local
Unpacking payload ...
Solving environment: | / - \ | / - \ | / - \ | / - \ | / - done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - _ipyw_jlab_nb_ext_conf==0.1.0=py37_0
    - _libgcc_mutex==0.1=main
    - alabaster==0.7.12=py37_0
    - anaconda-client==1.7.2=py37_0
    - anaco

/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Solving environment: - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - bravado
    - molvs
    - mordred


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    bravado-11.0.3             |     pyhd8ed1ab_0          31 KB  cond

In [5]:
!pip install molvs
!pip install bravado
!pip install umap

  Created wheel for strict-rfc3339: filename=strict_rfc3339-0.7-cp37-none-any.whl size=18120 sha256=18e3dfc71e32ac4a5a35fc49da86ec6fc0199283ea070f1d22fd86da303e052e
  Stored in directory: /root/.cache/pip/wheels/bb/af/c9/b6e9fb5f9b2470e4ed2a7241c9ab3a8cdd3bc8555ae02ca2e6
Successfully built strict-rfc3339
  Created wheel for umap: filename=umap-0.1.1-cp37-none-any.whl size=3568 sha256=0450eec186a892a412f02dca4ee2501fbe038e7c6a961889d0f1c2669ea9f2eb
  Stored in directory: /root/.cache/pip/wheels/7b/29/33/b4d917dc95f69c0a060e2ab012d95e15db9ed4cc0b94ccac26
Successfully built umap


In [6]:
import warnings
warnings.filterwarnings('ignore')

import json
import os
import requests

# Import AMPL libraries
# 
import atomsci.ddm.utils.data_curation_functions as dcf
import atomsci.ddm.utils.curate_data as curate_data
import atomsci.ddm.pipeline.diversity_plots as dp
import atomsci.ddm.pipeline.chem_diversity as cd
# Additional python libraries
import pandas as pd
import numpy as np
import getpass,os

In [48]:
! wget https://raw.githubusercontent.com/zp2226/E4511-2021-Zhanbo/main/Data/DRD2_ChEMBL_SMILES.csv
! wget https://raw.githubusercontent.com/zp2226/E4511-2021-Zhanbo/main/Data/DRD2_DTC_SMILES.csv
! wget https://raw.githubusercontent.com/zp2226/E4511-2021-Zhanbo/main/Data/DRD2_Excape_SMILES.csv

--2021-04-28 04:35:59--  https://raw.githubusercontent.com/zp2226/E4511-2021-Zhanbo/main/Data/DRD2_ChEMBL_SMILES.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11422321 (11M) [text/plain]
Saving to: ‘DRD2_ChEMBL_SMILES.csv.2’

DRD2_ChEMBL_SMILES. 100%[===================>]  10.89M  38.9MB/s    in 0.3s    

2021-04-28 04:36:00 (38.9 MB/s) - ‘DRD2_ChEMBL_SMILES.csv.2’ saved [11422321/11422321]

--2021-04-28 04:36:00--  https://raw.githubusercontent.com/zp2226/E4511-2021-Zhanbo/main/Data/DRD2_DTC_SMILES.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting 

In [112]:
chembl = pd.read_csv('DRD2_ChEMBL_SMILES.csv', header = 0)
chembl=chembl.drop(columns=chembl.iloc[:,0:1])
chembl

,Molecule_ChEMBL_ID,Molecule_Name,Molecule_Max_Phase,Molecular_Weight,#RO5_Violations,AlogP,Compound_Key,Smiles,Standard_Type,Standard_Relation,Standard_Value,Standard_Units,pChEMBL_Value,Data_Validity_Comment,Comment,Uo_Units,Ligand_Efficiency_BEI,Ligand_Efficiency_LE,Ligand_Efficiency_LLE,Ligand_Efficiency_SEI,Potential_Duplicate,Assay_ChEMBL_ID,Assay_Description,Assay_Type,BAO_Format_ID,BAO_Label,Assay_Organism,Assay_Tissue_ChEMBL_ID,Assay_Tissue_Name,Assay_Cell_Type,Assay_Subcellular_Fraction,Assay_Parameters,Assay_Variant_Accession,Assay_Variant_Mutation,Target_ChEMBL_ID,Target_Name,Target_Organism,Target_Type,Document_ChEMBL_ID,Source_ID,Source_Description,Document_Journal,Document_Year,Cell_ChEMBL_ID,Properties,Standard_Value_nM,base_rdkit_smiles
0,CHEMBL3697972,NaN,0.0,429.56,0.0,2.64,BDBM121475,O=C(CC1COC1)N[C@H]1CC[C@H](CCN2CCN(c3cccc4c3OC...,Ki,'=',105.930,nM,6.97,NaN,250041,UO_0000065,16.24,0.31,4.33,11.02,False,CHEMBL3707837,Radioligand Binding Assay: Aliquots of membran...,B,BAO_0000019,assay format,Homo sapiens,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CHEMBL217,Dopamine D2 receptor,Homo sapiens,SINGLE PROTEIN,CHEMBL3639169,37,BindingDB Database,NaN,2014.0,NaN,NaN,105.930,O=C(CC1COC1)N[C@H]1CC[C@H](CCN2CCN(c3cccc4c3OC...
1,CHEMBL3664863,NaN,0.0,414.59,0.0,4.71,BDBM107548,CCC(C)C(=O)N[C@H]1CC[C@H](CCN2CCC(c3cccc4c3OCO...,Ki,'=',351.610,nM,6.45,NaN,227177,UO_0000065,15.57,0.29,1.74,12.70,False,CHEMBL3707825,Biochemical Assay: The ability of the compound...,B,BAO_0000219,cell-based format,Homo sapiens,NaN,NaN,EBNA,NaN,NaN,NaN,NaN,CHEMBL217,Dopamine D2 receptor,Homo sapiens,SINGLE PROTEIN,CHEMBL3638396,37,BindingDB Database,NaN,2013.0,NaN,NaN,351.610,CCC(C)C(=O)N[C@H]1CC[C@H](CCN2CCC(c3cccc4c3OCO...
2,CHEMBL4564928,NaN,0.0,380.52,0.0,4.15,16a,c1ccc(Nc2nc(CN3CCOCC3)nc3sc4c(c23)CCCC4)cc1,pKb,'=',4.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,CHEMBL4309244,Displacement of [3H]Raclopride from recombinan...,B,BAO_0000219,cell-based format,Homo sapiens,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CHEMBL217,Dopamine D2 receptor,Homo sapiens,SINGLE PROTEIN,CHEMBL4308833,1,Scientific Literature,J Med Chem,2018.0,NaN,NaN,4.000,c1ccc(Nc2nc(CN3CCOCC3)nc3sc4c(c23)CCCC4)cc1
3,CHEMBL3318846,NaN,0.0,945.98,2.0,9.36,19,O=C(CCN1CCN(CCC(=O)OC2(c3ccc(Cl)cc3)CCN(CCCC(=...,Ki,'=',1.100,nM,8.96,NaN,NaN,UO_0000065,9.47,0.19,-0.40,8.99,False,CHEMBL3369122,Displacement of [3H]spiperone from human D2sho...,B,BAO_0000219,cell-based format,Homo sapiens,NaN,NaN,CHO,NaN,NaN,NaN,NaN,CHEMBL217,Dopamine D2 receptor,Homo sapiens,SINGLE PROTEIN,CHEMBL3352709,1,Scientific Literature,Bioorg. Med. Chem. Lett.,2014.0,CHEMBL3308072,NaN,1.100,O=C(CCN1CCN(CCC(=O)OC2(c3ccc(Cl)cc3)CCN(CCCC(=...
4,CHEMBL3322998,NaN,0.0,428.54,0.0,3.21,12,O=C(NCCCCN1CCN(c2ccnc3ccccc23)CC1)c1cn2ccccc2n1,Activity,NaN,NaN,NaN,NaN,NaN,Not Determined,NaN,NaN,NaN,NaN,NaN,False,CHEMBL3379902,Agonist activity at human D2R expressed in CHO...,B,BAO_0000219,cell-based format,Homo sapiens,NaN,NaN,CHO,NaN,NaN,NaN,NaN,CHEMBL217,Dopamine D2 receptor,Homo sapiens,SINGLE PROTEIN,CHEMBL3351252,1,Scientific Literature,J. Med. Chem.,2014.0,CHEMBL3308072,NaN,NaN,O=C(NCCCCN1CCN(c2ccnc3ccccc23)CC1)c1cn2ccccc2n1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20748,CHEMBL4589982,NaN,0.0,496.03,0.0,4.91,42,O=C(NCCCN1CCN(c2ccc(Cl)cc2)CC1)NN(Cc1ccc(F)cc1...,Ki,'=',500.000,nM,6.30,NaN,NaN,UO_0000065,12.70,0.25,1.39,12.39,False,CHEMBL4323478,Displacement of [3H] Spiperone from human D2L ...,B,BAO_0000219,cell-based format,Homo sapiens,NaN,NaN,CHO,NaN,NaN,NaN,NaN,CHEMBL217,Dopamine D2 receptor,Homo sapiens,SINGLE PROTEIN,CHEMBL4321823,1,Scientific Literature,J Med Chem,2019.0,CHEMBL3308072,NaN,500.000,O=C(NCCCN1CCN(c2ccc(Cl)cc2)CC1)NN(Cc1ccc(F)cc1...
20749,CHEMBL3890178,NaN,0.0,515.61,1.0,4.96,42a; single enantiomer 1,Cn1c(SCCCN2CC[C@]3(C[C@@H]3c3ccc(C(F)(F)F)cc3)...,Ki,'=',39.810,nM,7.40,NaN,Na

In [113]:
chembl['pRelation'] = chembl.Standard_Relation
chembl['pRelation'] = np.where(chembl.Standard_Relation == "'<'", ">", chembl.pRelation)
chembl['pRelation'] = np.where(chembl.Standard_Relation == "'>'", "<", chembl.pRelation)
chembl['pRelation'] = np.where(chembl.Standard_Relation == "'='", "=", chembl.pRelation)
chembl[['Standard_Relation', 'pRelation']].head()

,Standard_Relation,pRelation
0,'=',=
1,'=',=
2,'=',=
3,'=',=
4,NaN,NaN


In [114]:
chembl['pChEMBL_Value_nM'] = -np.log10(chembl.Standard_Value_nM/1000000000)

In [115]:
chembl=chembl.dropna(axis=0, subset = ['pChEMBL_Value_nM','base_rdkit_smiles'])


In [116]:
chembl.columns

Index(['Molecule_ChEMBL_ID', 'Molecule_Name', 'Molecule_Max_Phase',
       'Molecular_Weight', '#RO5_Violations', 'AlogP', 'Compound_Key',
       'Smiles', 'Standard_Type', 'Standard_Relation', 'Standard_Value',
       'Standard_Units', 'pChEMBL_Value', 'Data_Validity_Comment', 'Comment',
       'Uo_Units', 'Ligand_Efficiency_BEI', 'Ligand_Efficiency_LE',
       'Ligand_Efficiency_LLE', 'Ligand_Efficiency_SEI', 'Potential_Duplicate',
       'Assay_ChEMBL_ID', 'Assay_Description', 'Assay_Type', 'BAO_Format_ID',
       'BAO_Label', 'Assay_Organism', 'Assay_Tissue_ChEMBL_ID',
       'Assay_Tissue_Name', 'Assay_Cell_Type', 'Assay_Subcellular_Fraction',
       'Assay_Parameters', 'Assay_Variant_Accession', 'Assay_Variant_Mutation',
       'Target_ChEMBL_ID', 'Target_Name', 'Target_Organism', 'Target_Type',
       'Document_ChEMBL_ID', 'Source_ID', 'Source_Description',
       'Document_Journal', 'Document_Year', 'Cell_ChEMBL_ID', 'Properties',
       'Standard_Value_nM', 'base_rdkit_smiles'

In [117]:
chembl_cur =chembl[['Molecule_ChEMBL_ID','Smiles', 'Standard_Type', 'Standard_Relation', 'Standard_Value', 'pChEMBL_Value_nM','base_rdkit_smiles']]
chembl_cur.head()

,Molecule_ChEMBL_ID,Smiles,Standard_Type,Standard_Relation,Standard_Value,pChEMBL_Value_nM,base_rdkit_smiles
0,CHEMBL3697972,O=C(CC1COC1)N[C@H]1CC[C@H](CCN2CCN(c3cccc4c3OC...,Ki,'=',105.93,6.974981,O=C(CC1COC1)N[C@H]1CC[C@H](CCN2CCN(c3cccc4c3OC...
1,CHEMBL3664863,CCC(C)C(=O)N[C@H]1CC[C@H](CCN2CCC(c3cccc4c3OCO...,Ki,'=',351.61,6.453939,CCC(C)C(=O)N[C@H]1CC[C@H](CCN2CCC(c3cccc4c3OCO...
2,CHEMBL4564928,c1ccc(Nc2nc(CN3CCOCC3)nc3sc4c(c23)CCCC4)cc1,pKb,'=',4.00,8.397940,c1ccc(Nc2nc(CN3CCOCC3)nc3sc4c(c23)CCCC4)cc1
3,CHEMBL3318846,O=C(CCN1CCN(CCC(=O)OC2(c3ccc(Cl)cc3)CCN(CCCC(=...,Ki,'=',1.10,8.958607,O=C(CCN1CCN(CCC(=O)OC2(c3ccc(Cl)cc3)CCN(CCCC(=...
5,CHEMBL4463620,CCc1cc2c(cc1N1CCN(CC(O)CCNC(=O)c3cc4ccccc4[nH]...,Ki,'=',537.00,6.270026,CCc1cc2c(cc1N1CCN(CC(O)CCNC(=O)c3cc4ccccc4[nH]...


In [ ]:
chembl_cur = chembl_cur.loc[(chembl_cur.Standard_Type == 'IC50') | (chembl_cur.Standard_Type == 'EC50')]
chembl_cur=chembl_cur.rename(columns={'pChEMBL_Value_nM':'pXC50'})
chembl_cur = chembl_cur[~(chembl_cur.Molecule_ChEMBL_ID.isna())  &
            ~chembl_cur.Standard_Value.isna() &
             (chembl_cur.Standard_Relation == "'='" ) &
             ~chembl_cur.pXC50.isna()]
chembl_cur

In [119]:
thresh=-np.log10(100/1000000000)
chembl_cur = curate_data.aggregate_assay_data(chembl_cur, value_col = 'pXC50', 
                                              id_col = 'Molecule_ChEMBL_ID', 
                                              smiles_col = 'base_rdkit_smiles',
                                              relation_col = 'Standard_Relation',
                                              active_thresh=thresh)
chembl_cur

0 entries in input table are missing SMILES strings
1116 unique SMILES strings are reduced to 1116 unique base SMILES strings


,compound_id,base_rdkit_smiles,relation,pXC50,active
0,CHEMBL3394253,CCCN(CCCNC(=O)CCCC(=O)Oc1ccc(CCNc2nc(N)n3nc(-c...,,6.317577,0
1,CHEMBL1188090,CCCN(CCc1ccccc1)CCc1cccc(O)c1,,7.408935,1
2,CHEMBL3585373,CCCN(CCCNC(=O)CCC(=O)O)CCc1cccc2c1CC(=O)N2,,5.330016,0
3,CHEMBL633,CCCCc1oc2ccccc2c1C(=O)c1cc(I)c(OCCN(CC)CC)c(I)c1,,4.700014,0
4,CHEMBL3805165,CNC(=O)c1cccc(OCCCCN2CCN(c3cccc4c3OC(F)(F)O4)C...,,8.387216,1
...,...,...,...,...,...
1111,CHEMBL1086754,Cc1c(C(=O)NCCCN2CCN(c3cccc(Cl)c3Cl)CC2)cc(-c2c...,,5.594824,0
1112,CHEMBL2057451,COc1cc2c(cc1OC)[C@@H]1Cc3cc(C)cc(C)c3CN1CC2,,5.555121,0
1113,CHEMBL1202225,Clc1cc(Cl)cc(N2CCN(C[C@H]3C[C@@H]3c3ccccc3)CC2)c1,,7.494850,1
1114,CHEMBL1944691,O=c1[nH]c2c(O)ccc([C@@H](O)CNCCc3cccc(CNCCc4cc...,,6.399997,0


In [120]:
from google.colab import drive
drive.mount('/content/drive')




Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [121]:
chembl_cur.to_csv("/content/drive/MyDrive/DRD2_ChEMBL_cur.csv")

In [75]:
#DTC
dtc = pd.read_csv('DRD2_DTC_SMILES.csv', header = 0)
dtc=dtc.drop(columns=dtc.iloc[:,0:1])
dtc

,compound_id,standard_inchi_key,compound_name,synonym,target_id,target_pref_name,gene_names,wildtype_or_mutant,mutation_info,pubmed_id,standard_type,standard_relation,standard_value,standard_units,activity_comment,ep_action_mode,assay_format,assaytype,assay_subtype,inhibitor_type,detection_tech,assay_cell_line,compound_concentration_value,compound_concentration_value_unit,substrate_type,substrate_relation,substrate_value,substrate_units,assay_description,title,journal,doc_type,annotation_comments,pDTC_Value,SMILES,base_rdkit_smiles
0,CHEMBL8946,AADCDMQTJNYOSS-LBPRGKRZSA-N,ETICLOPRIDE,NaN,P14416,DOPAMINE D2 RECEPTOR,DRD2,NaN,NaN,1840645.0,KI,=,0.09,NM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.045757,CCC1=CC(=C(C(=C1O)C(=O)NC[C@@H]2CCCN2CC)OC)Cl,CCc1cc(Cl)c(OC)c(C(=O)NC[C@@H]2CCCN2CC)c1O
1,CHEMBL8946,AADCDMQTJNYOSS-LBPRGKRZSA-N,ETICLOPRIDE,NaN,P14416,DOPAMINE D2 RECEPTOR,DRD2,NaN,NaN,1840645.0,KI,=,2.10,NM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.677781,CCC1=CC(=C(C(=C1O)C(=O)NC[C@@H]2CCCN2CC)OC)Cl,CCc1cc(Cl)c(OC)c(C(=O)NC[C@@H]2CCCN2CC)c1O
2,CHEMBL8946,AADCDMQTJNYOSS-LBPRGKRZSA-N,ETICLOPRIDE,NaN,P14416,DOPAMINE D2 RECEPTOR,DRD2,NaN,NaN,2531826.0,KI,=,0.04,NM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.397940,CCC1=CC(=C(C(=C1O)C(=O)NC[C@@H]2CCCN2CC)OC)Cl,CCc1cc(Cl)c(OC)c(C(=O)NC[C@@H]2CCCN2CC)c1O
3,CHEMBL8946,AADCDMQTJNYOSS-LBPRGKRZSA-N,ETICLOPRIDE,NaN,P14416,DOPAMINE D2 RECEPTOR,DRD2,NaN,NaN,2869639.0,KI,=,0.31,NM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.508638,CCC1=CC(=C(C(=C1O)C(=O)NC[C@@H]2CCCN2CC)OC)Cl,CCc1cc(Cl)c(OC)c(C(=O)NC[C@@H]2CCCN2CC)c1O
4,CHEMBL8946,AADCDMQTJNYOSS-LBPRGKRZSA-N,ETICLOPRIDE,NaN,P14416,DOPAMINE D2 RECEPTOR,DRD2,NaN,NaN,2869639.0,KI,=,0.92,NM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.036212,CCC1=CC(=C(C(=C1O)C(=O)NC[C@@H]2CCCN2CC)OC)Cl,CCc1cc(Cl)c(OC)c(C(=O)NC[C@@H]2CCCN2CC)c1O
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17239,CHEMBL3819642,NaN,NaN,NaN,P14416,DOPAMINE D2 RECEPTOR,DRD2,NaN,NaN,27364609.0,Ki,=,758.58,NM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Displacement of [3H]N-methylspiperone from hum...,Development of CNS multi-receptor ligands: Mod...,Bioorg. Med. Chem.,PUBLICATION,NaN,6.119999,C1CN(CCC1(C2=CC=C(C=C2)Cl)O)CCCC3=NC4=CC=CC=C4...,OC1(c2ccc(Cl)cc2)CCN(CCCc2nc3ccccc3s2)CC1
17240,CHEMBL3819642,NaN,NaN,NaN,P14416,DOPAMINE D2 RECEPTOR,DRD2,NaN,NaN,27364609.0,Ki,=,763.00,NM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Displacement of [3H]N-methylspiperone from hum...,Development of CNS multi-receptor ligands: Mod...,Bioorg. Med. Chem.,PUBLICATION,NaN,6.117475,C1CN(CCC1(C2=CC=C(C=C2)Cl)O)CCCC3=NC4=CC=CC=C4...,OC1(c2ccc(Cl)cc2)CCN(CCCc2nc3ccccc3s2)CC1
17241,CHEMBL3823526,NaN,NaN,NaN,P14416,DOPAMINE D2 RECEPTOR,DRD2,NaN,NaN,NaN,Inhibition,=,18.00,%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Displacement of [3H]methyl-spiperone from huma...,In vivo phenotypic drug discovery: applying a ...,MedChemComm,PUBLICATION,NaN,18.000000,CC1=CSC(=C1C)C2C=CNC=N2.Cl,Cc1csc(C2C=CNC=N2)c1C
17242,CHEMBL3823832,NaN,NaN,NaN,P14416,DOPAMINE D2 RECEPTOR,DRD2,NaN,NaN,NaN,Inhibition,=,0.00,%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Displacement of [3H]methyl-spiperone from huma...,In vivo phenotypic drug discovery: applying a ...,MedChemComm,PUBLICATION,NaN,0.000000,CC1=C(C2=CC=CC=C2S1)[C@@H]3C=CNC=N3,Cc1sc2ccccc2c1[C@@H]1C=CNC=N1


In [93]:
vals = ['EC50', 'IC50', 'PIC50']
print(dtc.shape)
dtc=dtc[dtc.standard_type.isin(vals)]
print(dtc.shape)
dtc['pRelation'] = dtc.standard_relation
dtc['pRelation'] = np.where(dtc.standard_relation == "<", ">", dtc.pRelation)
dtc['pRelation'] = np.where(dtc.standard_relation == ">", "<", dtc.pRelation)
dtc['pRelation'] = np.where(dtc.standard_relation == "=", "=", dtc.pRelation)
dtc.pRelation
display(dtc[['standard_relation', 'pRelation']].head())
dtc=dtc.dropna(axis=0, subset = ['pDTC_Value','base_rdkit_smiles'])
print(dtc.shape)
dtc_cur = curate_data.aggregate_assay_data(dtc, value_col = 'pDTC_Value', 
                                              output_value_col = 'pXC50',
                                              id_col = 'compound_id', 
                                              smiles_col = 'base_rdkit_smiles',
                                              relation_col = 'pRelation',
                                              active_thresh=thresh)
dtc_cur.to_csv("/content/drive/MyDrive/DRD2_DTC_cur.csv")


(1510, 37)
(1510, 37)


,standard_relation,pRelation
26,=,=
27,=,=
34,=,=
56,>,<
59,>,<


(1510, 37)
0 entries in input table are missing SMILES strings
1069 unique SMILES strings are reduced to 1069 unique base SMILES strings


In [94]:
print(dtc_cur.shape)
print(dtc_cur.relation.unique())
dtc_cur.head()

(1069, 5)
['' '>' '<']


,compound_id,base_rdkit_smiles,relation,pXC50,active
0,CHEMBL3394253,CCCN(CCCNC(=O)CCCC(=O)Oc1ccc(CCNc2nc(N)n3nc(-c...,,6.317577,0
1,CHEMBL3585373,CCCN(CCCNC(=O)CCC(=O)O)CCc1cccc2c1CC(=O)N2,,5.330016,0
2,CHEMBL633,CCCCc1oc2ccccc2c1C(=O)c1cc(I)c(OCCN(CC)CC)c(I)c1,,4.700014,0
3,CHEMBL1188090,CCCN(CCc1ccccc1)CCc1cccc(O)c1,,7.408935,1
4,CHEMBL571858,CN(C)CCn1cc(S(=O)(=O)c2cccc(F)c2)c2cccnc21,>,5.301030,0


(17244, 36)


AttributeError: ignored

In [101]:
excape = pd.read_csv('DRD2_Excape_SMILES.csv', header = 0)
excape=excape.drop(columns=excape.iloc[:,0:1])
excape

,Ambit_InchiKey,Original_Entry_ID,Entrez_ID,Activity_Flag,pXC50,DB,Original_Assay_ID,Tax_ID,Gene_Symbol,Ortholog_Group,SMILES,base_rdkit_smiles
0,AZKHPPMTYXHBSH-PWQYDOHDNA-N,660059,1813,N,NaN,pubchem_screening,485358,9606,DRD2,1171,OC=1N=C(NCCCCCC(=O)O)N=C2N=CN(C21)CC3=CC=CC=C3,O=C(O)CCCCCNc1nc(O)c2c(ncn2Cc2ccccc2)n1
1,AZKKMMRAFWUBCN-UHFFFAOYNA-N,CHEMBL1834261,1813,A,5.24,chembl20,771231,9606,DRD2,1171,N1(CCN(CC1)CCCCC=2C=CC(=CC2)OC)CC=3OC=4C(C3)=C...,COc1ccc(CCCCN2CCN(Cc3cc4ccccc4o3)CC2)cc1
2,AZKLAVZPMHYNOT-DDEWIEPONA-N,2409129,1813,N,NaN,pubchem_screening,485358,9606,DRD2,1171,O=C(/C=C/1\N(C=2C(C1(C)C)=CC=CC2)C)COC3=CC(NC(...,CC(=O)Nc1cccc(OCC(=O)/C=C2\N(C)c3ccccc3C2(C)C)c1
3,AZKNBPNSQOVWPK-UHFFFAOYNA-N,5308350,1813,N,NaN,pubchem_screening,485358,9606,DRD2,1171,S(O)(O)(=CC(=O)N1CCC=2C1=CC=CC2)=CC=3N=C(OC3C)...,Cc1oc(-c2ccccc2F)nc1C=S(O)(O)=CC(=O)N1CCc2ccccc21
4,AZKOPRBLDAIRFJ-UHFFFAOYNA-N,16315772,1813,N,NaN,pubchem_screening,485358,9606,DRD2,1171,S(=O)(=O)(N1CCC(CC1)C(=O)N(CC2=CC(F)=C(OC)C=C2...,COc1ccc(CN(C)C(=O)C2CCN(S(=O)(=O)c3cccs3)CC2)cc1F
...,...,...,...,...,...,...,...,...,...,...,...,...
76615,HWNCNYMVSQNVOB-UHFFFAOYNA-N,3209083,1813,N,NaN,pubchem_screening,485358,9606,DRD2,1171,N1(CCN(CC1)CC=2N(N=NN2)C(CC)(C)C)C(C3=CC=CC=C3...,CCC(C)(C)n1nnnc1CN1CCN(C(c2ccccc2)c2ccccc2)CC1
76616,HWNCPNFBIMSDEA-TXSGWPFENA-N,1945070,1813,N,NaN,pubchem_screening,485358,9606,DRD2,1171,S=C(NC(=O)C=1C=CC(F)=CC1)NNC(=O)C=2OC=CC2,O=C(NC(=S)NNC(=O)c1ccco1)c1ccc(F)cc1
76617,HWNGUFFTLMHXOE-UHFFFAOYNA-N,15945059,1813,N,NaN,pubchem_screening,485347,9606,DRD2,1171,O1C2=C(C(OC)=CC=C2C=CC1=O)C(=O)C=C(C)C,COc1ccc2ccc(=O)oc2c1C(=O)C=C(C)C
76618,HWNJOJRSDAWOAC-UHFFFAOYNA-N,16187540,1813,N,NaN,pubchem_screening,485358,9606,DRD2,1171,ClC=1C=CC(C(=O)C2CN(CCC2)CC3=CC(=C(OC)C=C3)CN4...,COc1ccc(CN2CCCC(C(=O)c3ccc(Cl)cc3)C2)cc1Cn1cncn1


In [103]:
excape['pRelation']=''
print(excape.shape)
print(excape.pRelation.unique())
excape=excape.dropna(axis=0, subset = ['pXC50','base_rdkit_smiles'])
print(excape.shape)
# aggregate duplicates
excape_cur = curate_data.aggregate_assay_data(excape, value_col = 'pXC50', 
                                              output_value_col = 'pXC50',
                                              id_col = 'Original_Entry_ID', 
                                              smiles_col = 'base_rdkit_smiles',
                                              relation_col = 'pRelation',
                                              active_thresh=thresh)


(1046, 13)
['']
(1046, 13)
0 entries in input table are missing SMILES strings
1045 unique SMILES strings are reduced to 1045 unique base SMILES strings


In [105]:
print(excape_cur.shape)
print(excape_cur.relation.unique())
excape_cur.head()
excape_cur.to_csv("/content/drive/MyDrive/DRD2_EXCAPE_cur.csv")

(1045, 5)
['']


In [129]:
chembl_cur

,compound_id,base_rdkit_smiles,relation,pXC50,active
0,CHEMBL3394253,CCCN(CCCNC(=O)CCCC(=O)Oc1ccc(CCNc2nc(N)n3nc(-c...,,6.317577,0
1,CHEMBL1188090,CCCN(CCc1ccccc1)CCc1cccc(O)c1,,7.408935,1
2,CHEMBL3585373,CCCN(CCCNC(=O)CCC(=O)O)CCc1cccc2c1CC(=O)N2,,5.330016,0
3,CHEMBL633,CCCCc1oc2ccccc2c1C(=O)c1cc(I)c(OCCN(CC)CC)c(I)c1,,4.700014,0
4,CHEMBL3805165,CNC(=O)c1cccc(OCCCCN2CCN(c3cccc4c3OC(F)(F)O4)C...,,8.387216,1
...,...,...,...,...,...
1111,CHEMBL1086754,Cc1c(C(=O)NCCCN2CCN(c3cccc(Cl)c3Cl)CC2)cc(-c2c...,,5.594824,0
1112,CHEMBL2057451,COc1cc2c(cc1OC)[C@@H]1Cc3cc(C)cc(C)c3CN1CC2,,5.555121,0
1113,CHEMBL1202225,Clc1cc(Cl)cc(N2CCN(C[C@H]3C[C@@H]3c3ccccc3)CC2)c1,,7.494850,1
1114,CHEMBL1944691,O=c1[nH]c2c(O)ccc([C@@H](O)CNCCc3cccc(CNCCc4cc...,,6.399997,0


In [128]:
dtc_cur

,compound_id,base_rdkit_smiles,relation,pXC50,active
0,CHEMBL3394253,CCCN(CCCNC(=O)CCCC(=O)Oc1ccc(CCNc2nc(N)n3nc(-c...,,6.317577,0
1,CHEMBL3585373,CCCN(CCCNC(=O)CCC(=O)O)CCc1cccc2c1CC(=O)N2,,5.330016,0
2,CHEMBL633,CCCCc1oc2ccccc2c1C(=O)c1cc(I)c(OCCN(CC)CC)c(I)c1,,4.700014,0
3,CHEMBL1188090,CCCN(CCc1ccccc1)CCc1cccc(O)c1,,7.408935,1
4,CHEMBL571858,CN(C)CCn1cc(S(=O)(=O)c2cccc(F)c2)c2cccnc21,>,5.301030,0
...,...,...,...,...,...
1064,CHEMBL1086754,Cc1c(C(=O)NCCCN2CCN(c3cccc(Cl)c3Cl)CC2)cc(-c2c...,,5.594824,0
1065,CHEMBL487788,O=C(Nc1cccc(SC(F)(F)F)c1)N1CCC(N2CCC(n3c(=O)[n...,,5.958607,0
1066,CHEMBL3299093,COc1ccccc1N1CCN(CCNC(=O)OC(C)(C)C)CC1,,7.030049,1
1067,CHEMBL1944691,O=c1[nH]c2c(O)ccc([C@@H](O)CNCCc3cccc(CNCCc4cc...,,6.399997,0


In [127]:
excape_cur

,compound_id,base_rdkit_smiles,relation,pXC50,active
0,73057094,Oc1cccc2c1C[C@H]1OCCN(CC3CC3)[C@@H]1C2,,7.74473,1
1,46893146,COc1ccccc1N1CCN(CCCCN2CCc3cc(-c4cccs4)ccc3C2=O...,,8.16115,1
2,CHEMBL600986,CCS(=O)(=O)Nc1ccc2c(c1)CC[C@H]1[C@H]2c2cc(O)c(...,,5.48000,0
3,CHEMBL560715,CC(C)N1CCN(CCN2CCN(c3cccc(Cl)c3)C2=O)CC1,,6.70000,0
4,44214765,Nc1cccc(-c2ccc(C(=O)NC3CCN(Cc4ccccc4)C3)cc2)c1,,5.30103,0
...,...,...,...,...,...
1040,CHEMBL2057455,COc1cc2c(c(OC)c1)CN1CCc3cc(OC)c(O)cc3[C@@H]1C2,,5.60000,0
1041,CHEMBL2057451,COc1cc2c(cc1OC)[C@@H]1Cc3cc(C)cc(C)c3CN1CC2,,6.65000,0
1042,CHEMBL111749,CNc1cc(OC)c(C(=O)N[C@H]2CN(Cc3ccccc3)C[C@@H]2C...,,7.96000,1
1043,90645296,COc1ccccc1N1CCN(CCNC(=O)OC(C)(C)C)CC1,,7.03012,1


In [123]:
DRD2 = pd.concat([chembl_cur, dtc_cur, excape_cur])


In [126]:
DRD2.to_csv("/content/drive/MyDrive/DRD2_Merge.csv")
print(DRD2.shape)

(3230, 5)


In [132]:

DRD2.active.value_counts()
DRD2.isna().sum()

compound_id          0
base_rdkit_smiles    0
relation             0
pXC50                0
active               0
dtype: int64